In [98]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [99]:
df = pd.read_csv("/kaggle/input/flickr8kimagescaptions/flickr8k/captions.txt")

In [100]:
df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [101]:
list_dir = '/kaggle/input/flickr8kimagescaptions/flickr8k/images'
#this is where i am going the images to get the features

In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
# from torchsummary import summary

In [103]:
# Load pre-trained AlexNet without the final fully connected layer
vgg = models.vgg16(pretrained=True)
vgg = nn.Sequential(*list(vgg.children())[:-1])  # Remove the last fully connected layer

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 177MB/s]  


In [104]:
print(vgg)
vgg.eval()

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [105]:
import torchvision.transforms as transforms
from PIL import Image

# Define a transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),             # Resize to a larger size
    transforms.CenterCrop(224),         # Crop the center to 224x224
    transforms.ToTensor(),              # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [107]:
import os
features = {}
# Get the list of file names in the directory
image_files = os.listdir(list_dir)
# Iterate over the image files
for image_file in image_files:
    # Check if the file is an image (you can add more file format checks if needed)
    if image_file.endswith('.jpg') or image_file.endswith('.png'):
        # Construct the full path to the image file
        image_path = os.path.join(list_dir, image_file)
        image = Image.open(image_path)
        image_tensor = preprocess(image)
        image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension
        
        # Extract features using VGG16
        with torch.no_grad():
            print(image_file)
            pred = vgg(image_tensor)
            features[image_file] = pred.numpy()
        # Now you can do something with the image_path, for example, load and process the image
        # Your code to process each image goes here
#         print(f"Processing image: {image_path}")

3226254560_2f8ac147ea.jpg
214543992_ce6c0d9f9b.jpg
2366643786_9c9a830db8.jpg
3368819708_0bfa0808f8.jpg
2190227737_6e0bde2623.jpg
2752809449_632cd991b3.jpg
3097776588_312932e438.jpg
1206506157_c7956accd5.jpg
1319634306_816f21677f.jpg
2465218087_fca77998c6.jpg
3351493005_6e5030f596.jpg
2949337912_beba55698b.jpg
534886684_a6c9f40fa1.jpg
3543600125_223747ef4c.jpg
2174206711_11cb712a8d.jpg
280706862_14c30d734a.jpg
2100735137_05c6079537.jpg
3131632154_098f86f4cb.jpg
2236016316_f476cbbf06.jpg
3335375223_b4da8df523.jpg
929679367_ff8c7df2ee.jpg
241347214_5f19e7998c.jpg
2998024845_1529c11694.jpg
1317292658_ba29330a0b.jpg
2521938720_911ac092f7.jpg
3426933951_2302a941d8.jpg
2410399168_1462c422d4.jpg
2587106431_1cc0e719c6.jpg
2339516180_12493e8ecf.jpg
1827560917_c8d3c5627f.jpg
2623247254_3bfc795121.jpg
2813588204_69fe7deb14.jpg
3396043950_12783c5147.jpg
3356284586_21c6f155a5.jpg
618771382_459bb492e4.jpg
3411393875_a9ff73c67a.jpg
3046431231_dc48851062.jpg
3126795109_73920ed5dc.jpg
2199200615_85e4c2a

In [112]:
import tqdm
import os


In [113]:
import pickle
out_dir = '/kaggle/working/'
pickle.dump(features,open(os.path.join(out_dir,'features.pkl'), 'wb'))

In [114]:
pickle_file_path = '/kaggle/working/features.pkl'

# Load the data from the pickle file
with open(pickle_file_path, 'rb') as file:
    features = pickle.load(file)

In [115]:
cap_dir = '/kaggle/input/flickr8kimagescaptions/flickr8k/'
captions = {}
with open(os.path.join(cap_dir,'captions.txt'), 'rb') as f:
    next(f)
    captions_doc = f.read()

In [116]:
from tqdm.notebook import tqdm

In [117]:
for line in tqdm(captions_doc.decode('utf-8').split('\n')):
    if(len(line) < 2):
        continue
    token = line.split(',')
    image_id, caption = token[0], token[1]
    img = image_id.split('.')
    image = img[0]
    caption = " ".join(caption)
    if(image not in captions):
        captions[image] = []
    captions[image].append(caption)
print("done!")

  0%|          | 0/40456 [00:00<?, ?it/s]

done!


In [118]:
mapping = {}
# process lines
for line in tqdm(captions_doc.decode('utf-8').split('\n')):
    # split the line by comma(,)
    tokens = line.split(',')
    if len(line) < 2:
        continue
    image_id, caption = tokens[0], tokens[1:]
    # remove extension from image ID
    image_id = image_id.split('.')[0]
    # convert caption list to string
    caption = " ".join(caption)
    # create list if needed
    if image_id not in mapping:
        mapping[image_id] = []
    # store the caption
    mapping[image_id].append(caption)

  0%|          | 0/40456 [00:00<?, ?it/s]

In [119]:
len(mapping)

8091

In [120]:
import re
def clean(captions):
#     cleaned_captions = {}
    for key, caption_list in captions.items():
#         cleaned_captions[key] = []
        for i in range(len(caption_list)):
            cap = caption_list[i]
            cap = cap.lower()
            # delete digits, special chars, etc., 
            cap = cap.replace('[^A-Za-z]', '')
            # delete additional spaces
            cap = cap.replace('\s+', ' ')
            # add start and end tags to the caption
            cap = 'startseq ' + " ".join([word for word in cap.split() if len(word)>1]) + ' endseq'
            caption_list[i] = cap

In [121]:
clean(mapping)

In [122]:
all_captions = []
for keys in mapping:
    for cap in mapping[keys]:
        all_captions.append(cap)
len(all_captions)

40455

In [177]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Choose a tokenizer (you can use different tokenizers based on your requirements)
tokenizer = get_tokenizer("basic_english")

# Tokenize each sentence in the list
tokenized_sentences = [tokenizer(sentence) for sentence in all_captions]

# Create a vocabulary based on the tokenized sentences, including '<unk>'
vocab = build_vocab_from_iterator(tokenized_sentences, specials=['<unk>'])

In [178]:
tokenizer

<function torchtext.data.utils._basic_english_normalize(line)>

In [179]:
vocab_size = len(vocab)
max_length = max(len(tokens) for tokens in tokenized_sentences)
print(vocab_size)
print(max_length)

8897
35


In [180]:
image_ids = list(mapping.keys())
split = int(len(image_ids)*0.90)
train = image_ids[:split]
test = image_ids[split:]

In [181]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, data_keys, mapping, features, vocab, max_length):
        self.data_keys = data_keys
        self.mapping = mapping
        self.features = features
        self.vocab = vocab
        self.max_length = max_length

    def __len__(self):
        return len(self.data_keys)

    def __getitem__(self, idx):
        key = self.data_keys[idx]
        captions = self.mapping[key]
        X1, X2, y = list(), list(), list()

        for caption in captions:
            seq = [self.vocab[word] if word in self.vocab else self.vocab['<unk>'] for word in caption.split()]  # Use '<unk>' for out-of-vocabulary words
            print(seq)
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = torch.tensor(in_seq, dtype=torch.long)  # Convert list of indices to tensor
                out_seq = torch.tensor(out_seq, dtype=torch.long)  # Convert single index to tensor

                X1.append(self.features[key + '.jpg'][0])
                X2.append(in_seq)
                y.append(out_seq)

        return torch.tensor(X1), torch.nn.utils.rnn.pad_sequence(X2, batch_first=True), torch.stack(y)


def data_generator(data_keys, mapping, features, vocab, max_length, vocab_size, batch_size):
    custom_dataset = CustomDataset(data_keys, mapping, features,vocab, max_length)
    data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)
    
    for batch in data_loader:
        X1, X2, y = batch
        yield [X1, X2], y

In [182]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )

    def forward(self, x):
        return self.fc(x)

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.dropout = nn.Dropout(0.4)
        self.lstm = nn.LSTM(embedding_size, hidden_size)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.dropout(embedded)
        lstm_output, _ = self.lstm(embedded)
        return self.fc(lstm_output[-1])

class ImageCaptioningModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, image_input_size, max_length):
        super(ImageCaptioningModel, self).__init__()
        self.encoder = Encoder(input_size=image_input_size, hidden_size=256)
        self.decoder = Decoder(vocab_size=vocab_size, embedding_size=256, hidden_size=256)

    def forward(self, image_input, caption_input):
        encoded_image = self.encoder(image_input)
        decoded_caption = self.decoder(caption_input)
        return decoded_caption

# Assuming you have the necessary parameters like vocab_size, embedding_size, hidden_size, etc.
vocab_size = 8896
embedding_size = 256
hidden_size = 256
image_input_size = 4096
max_length = 35

# Create an instance of the model
model = ImageCaptioningModel(vocab_size, embedding_size, hidden_size, image_input_size, max_length)

# Display the model architecture
print(model)

ImageCaptioningModel(
  (encoder): Encoder(
    (fc): Sequential(
      (0): Dropout(p=0.4, inplace=False)
      (1): Linear(in_features=4096, out_features=256, bias=True)
      (2): ReLU()
    )
  )
  (decoder): Decoder(
    (embedding): Embedding(8896, 256, padding_idx=0)
    (dropout): Dropout(p=0.4, inplace=False)
    (lstm): LSTM(256, 256)
    (fc): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=8896, bias=True)
      (3): Softmax(dim=1)
    )
  )
)


In [183]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [184]:
epochs = 20
batch_size = 64
vocab_size = 8896
embedding_size = 256
hidden_size = 256
image_input_size = 4096
max_length = 35

for epoch in range(epochs):
    # Create data loader
    data_loader = data_generator(train, mapping, features, vocab, max_length, vocab_size, batch_size)
    
    # Set the model to training mode
    model.train()

    # Training for one epoch
    for batch in data_loader:
        image_input, sequence_input, target = batch

        # Forward pass
        outputs = model(image_input, sequence_input)

        # Compute the loss
        loss = criterion(outputs, target)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}")

[2, 14, 7, 26, 8, 45, 3, 244, 68, 51, 1]
[2, 14, 8, 2004, 3, 4, 44, 1]
[2, 14, 8, 32, 1]
[2, 45, 14, 8, 1]
[2, 4, 14, 8, 248, 522, 11, 6230, 79, 245, 75, 4, 67, 1]
[2, 15, 6, 3, 244, 137, 135, 158, 21, 530, 119, 35, 12, 99, 7, 291, 183, 101, 149, 1]
[2, 15, 20, 30, 234, 6, 366, 1]
[2, 2274, 366, 21, 530, 119, 1]
[2, 185, 3, 137, 158, 3, 530, 119, 1]
[2, 25, 10, 229, 419, 5, 29, 135, 1]
[2, 4, 48, 31, 16, 32, 100, 68, 4, 288, 1]
[2, 48, 31, 16, 34, 147, 5, 4, 388, 288, 11, 179, 1]
[2, 48, 31, 87, 54, 179, 1]
[2, 48, 31, 87, 68, 4, 23, 1]
[2, 12, 570, 31, 808, 884, 8, 100, 435, 1]
[2, 4, 12, 506, 3, 30, 234, 7, 380, 456, 174, 84, 3081, 1]
[2, 12, 64, 2589, 7487, 493, 3081, 9, 1234, 3706, 5, 206, 1]
[2, 12, 25, 64, 6179, 1234, 3081, 1]
[2, 12, 25, 64, 7714, 93, 5181, 20, 1086, 1441, 7, 294, 19, 2589, 3081, 1]
[2, 12, 25, 64, 116, 3, 47, 11, 224, 1557, 5181, 1]
[2, 64, 5, 449, 2719, 93, 500, 21, 6939, 1]
[2, 64, 211, 5, 449, 35, 136, 500, 1]
[2, 64, 77, 449, 136, 500, 3, 4612, 2624, 1]
[2,

RuntimeError: stack expects each tensor to be equal size, but got [38, 512, 7, 7] at entry 0 and [48, 512, 7, 7] at entry 1

In [164]:
# features['2698614194_b4e6e11dff.jpg'][0]